# Gradio

In [4]:
#gradio 설치
# pip install gradio

SyntaxError: invalid syntax (225969832.py, line 2)

## 시도 1. Gradio를 사용하여 이미지에 세피아 효과를 적용하는 인터페이스를 생성

In [5]:
import numpy as np
import gradio as gr

def sepia(input_img):
    sepia_filter = np.array([
        [0.393, 0.769, 0.189], 
        [0.349, 0.686, 0.168], 
        [0.272, 0.534, 0.131]
    ])
    sepia_img = input_img.dot(sepia_filter.T)
    sepia_img /= sepia_img.max()
    return sepia_img


demo = gr.Interface(sepia, gr.Image(shape=(200, 200)), "image") #gr.Image(type="filepath", shape=...)
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://89504f1b43285e8021.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## 시도 2. Gradio를 사용하여 이미지에 웜톤 필터를 적용하는 인터페이스를 생성

In [ ]:
import numpy as np
import gradio as gr
from PIL import Image

def tone_filter(input_img):
    input_img = input_img.convert("RGB")  # 이미지를 RGB로 변환
    input_arr = np.array(input_img)  # PIL.Image 객체를 NumPy 배열로 변환
    hsv_arr = np.array(input_img.convert("HSV"))  # HSV로 변환한 NumPy 배열
    hue = hsv_arr[:, :, 0]
    is_warm_tone = np.logical_or(np.logical_and(hue > 0, hue < 60), np.logical_and(hue > 300, hue <= 360))
    filtered_arr = np.zeros_like(input_arr)
    filtered_arr[is_warm_tone] = input_arr[is_warm_tone]
    filtered_img = Image.fromarray(filtered_arr)  # NumPy 배열을 PIL.Image 객체로 변환
    return filtered_img

iface = gr.Interface(fn=tone_filter, inputs="image", outputs="image")
iface.launch(share='True')


 - tone_filter 함수는 입력 이미지에 웜톤 필터를 적용하여 웜톤 이미지를 생성한다.

 - 주어진 이미지를 RGB로 변환하고, 이를 NumPy 배열로 변환한 후 HSV로 변환한다.

 - HSV 배열에서 색상(Hue) 채널을 추출하여 웜톤 여부를 확인할 수 있다.
 
 - 하지만, NumPy 배열 객체에는 'convert'라는 속성(attribute)이 없어서 발생하는 오류로 실행이 안된다.

## 시도3. StyleTransferModel이라는 사용자 정의 스타일 변환 모델을 정의하고 수행
 - 이 모델은 사전 훈련된 VGG19 모델의 일부 레이어를 사용하여 스타일 변환을 수행

In [ ]:
import gradio as gr
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as transforms
from torchvision.models import vgg19
from torchvision.models.utils import load_state_dict_from_url

# 스타일 변환 모델 정의
class StyleTransferModel(torch.nn.Module):
    def __init__(self):
        super(StyleTransferModel, self).__init__()
        self.content_layers = ['conv_4']
        self.style_layers = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']
        self.model = self.load_model()

    def load_model(self):
        model = vgg19(pretrained=False)
        state_dict = load_state_dict_from_url('https://download.pytorch.org/models/vgg19-dcbb9e9d.pth', progress=True)
        model.load_state_dict(state_dict)

        return model.features[:6]

    def forward(self, content, style):
        content_features = self.model(content)
        style_features = self.model(style)

        return content_features, style_features

# 스타일 변환 함수
def style_transfer(content_image, style_image):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    content_image = Image.fromarray(content_image)
    style_image = Image.fromarray(style_image)

    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor()
    ])

    content_tensor = transform(content_image).unsqueeze(0).to(device)
    style_tensor = transform(style_image).unsqueeze(0).to(device)

    model = StyleTransferModel().to(device)
    content_features, style_features = model(content_tensor, style_tensor)

    # 여기에서 스타일 변환 작업을 수행하고 결과 이미지를 생성합니다.
    # 예시로서 content_features와 style_features를 이용하여 다른 이미지를 생성하도록 작성해두었습니다.
    # 실제로는 원하는 스타일 변환 알고리즘 또는 모델로 대체해야 합니다.
    output_image = content_features + style_features  # 예시로서 content_features와 style_features를 더한 결과를 생성

    output_image_np = output_image.squeeze(0).detach().cpu().numpy()  # NumPy 배열로 변환

    # 결과 이미지를 PIL.Image 객체로 변환하여 반환합니다.
    output_image_pil = Image.fromarray((output_image_np * 255).astype(np.uint8))

    return output_image_pil

iface = gr.Interface(fn=style_transfer, inputs=["image", "image"], outputs="image")
iface.launch(share='True')


In [ ]:
import gradio as gr
import numpy as np
import torch
import torchvision.transforms as transforms
from torchvision.models import vgg19, vgg19_bn
from torchvision.models.utils import load_state_dict_from_url

def style_transfer(content_image, style_image):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 전처리: 이미지 크기 조정 및 텐서로 변환
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    content_tensor = transform(content_image).unsqueeze(0).to(device)
    style_tensor = transform(style_image).unsqueeze(0).to(device)

    # 사전 훈련된 VGG19 모델 로드
    model = vgg19(pretrained=True).features.to(device).eval()

    # 스타일 변환 수행
    with torch.no_grad():
        content_features = model(content_tensor)
        style_features = model(style_tensor)

        # 여기에서 스타일 변환 작업을 수행하고 결과 이미지를 생성합니다.
        # 예를 들어, content_features와 style_features를 이용하여 다른 이미지를 생성합니다.
        output_features = content_features + style_features

    # 후처리: 텐서를 이미지로 변환
    output_image = output_features.squeeze(0).cpu().numpy()
    output_image = output_image.transpose((1, 2, 0))
    output_image = np.clip(output_image, 0, 1)  # 범위를 0과 1 사이로 클리핑
    output_image = (output_image * 255).astype(np.uint8)

    return output_image

iface = gr.Interface(fn=style_transfer, inputs=["image", "image"], outputs="image")
iface.launch(import gradio as gr
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as transforms
from torchvision.models import vgg19
from torchvision.models.utils import load_state_dict_from_url

# 스타일 변환 모델 정의
class StyleTransferModel(torch.nn.Module):
    def __init__(self):
        super(StyleTransferModel, self).__init__()
        self.content_layers = ['conv_4']
        self.style_layers = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']
        self.model = self.load_model()

    def load_model(self):
        model = vgg19(pretrained=False)
        state_dict = load_state_dict_from_url('https://download.pytorch.org/models/vgg19-dcbb9e9d.pth', progress=True)
        model.load_state_dict(state_dict)

        return model.features[:6]

    def forward(self, content, style):
        content_features = self.model(content)
        style_features = self.model(style)

        return content_features, style_features

# 스타일 변환 함수
def style_transfer(content_image, style_image):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    content_image = Image.fromarray(content_image)
    style_image = Image.fromarray(style_image)

    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor()
    ])

    content_tensor = transform(content_image).unsqueeze(0).to(device)
    style_tensor = transform(style_image).unsqueeze(0).to(device)

    model = StyleTransferModel().to(device)
    content_features, style_features = model(content_tensor, style_tensor)

    # 여기에서 스타일 변환 작업을 수행하고 결과 이미지를 생성합니다.
    # 예시로서 content_features와 style_features를 이용하여 다른 이미지를 생성하도록 작성해두었습니다.
    # 실제로는 원하는 스타일 변환 알고리즘 또는 모델로 대체해야 합니다.
    output_image = content_features + style_features  # 예시로서 content_features와 style_features를 더한 결과를 생성

    output_image_np = output_image.squeeze(0).detach().cpu().numpy()  # NumPy 배열로 변환

    # 결과 이미지를 PIL.Image 객체로 변환하여 반환합니다.
    output_image_pil = Image.fromarray((output_image_np * 255).astype(np.uint8))

    return output_image_pil

iface = gr.Interface(fn=style_transfer, inputs=["image", "image"], outputs="image")
iface.launch(share='True')
)
